<a href="https://colab.research.google.com/github/sayandas96476/RAG/blob/main/Sparse_retrieval_wikipedia_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install -q langchain-groq langchain



In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
import requests
from bs4 import BeautifulSoup

from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import os
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [41]:

def get_full_wikipedia_content(title):
    # Construct the URL for the full Wikipedia page
    url = f"https://en.wikipedia.org/wiki/{title.replace(' ', '_')}"

    try:
        # Make the request
        response = requests.get(url)
        response.raise_for_status()

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the main content div
        content_div = soup.find(id="mw-content-text")

        # Extract all paragraphs
        paragraphs = content_div.find_all('p')

        # Combine all paragraph texts
        full_text = '\n\n'.join([para.get_text() for para in paragraphs])

        # Remove citations [1], [2], etc.
        import re
        full_text = re.sub(r'\[\d+\]', '', full_text)

        return full_text.strip()

    except requests.RequestException as e:
        return f"Error fetching page: {str(e)}"

# First install beautifulsoup4 if you haven't:
# pip install beautifulsoup4

# Example usage


In [42]:

def combine_strings(original_list, chunk_size=3):
    return [''.join(original_list[i:i + chunk_size])
            for i in range(0, len(original_list), chunk_size)]



In [43]:

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)



In [44]:
title = "Batman"
text = get_full_wikipedia_content(title)
text = text.replace('\n', '')

lis = text.split(".")

combined = combine_strings(lis)
text = """ """
for i in combined:
  text += i+"\n\n\n"

documents = text.split("\n\n\n")
len(documents)

preprocessed_docs = [preprocess_text(doc) for doc in documents]

In [45]:

# Create a ChatGroq instance
llm = ChatGroq(
    model_name="mixtral-8x7b-32768",  # You can also use "llama2-70b-4096"
    temperature=0.7,
    max_tokens=1024
)

In [46]:
def get_context(preprocessed_docs,query):

  # Step 2: Initialize the TfidfVectorizer
  vectorizer = TfidfVectorizer()

  # Step 3: Fit and transform the documents to generate the TF-IDF matrix
  tfidf_matrix = vectorizer.fit_transform(preprocessed_docs)




  # Step 5: Transform the query into a TF-IDF vector
  query_vector = vectorizer.transform([query])

  # Step 6: Compute cosine similarity between the query and the documents
  cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

  # Step 7: Get the top 3 results based on similarity scores
  top_n = 10
  top_indices = np.argsort(cosine_similarities)[::-1][:top_n]

  CONTEXT = """"""

  for i, idx in enumerate(top_indices):
      CONTEXT+= documents[idx] + "\n"+ "================"+"\n"
  return CONTEXT

In [47]:
def retrieve(CONTEXT,query):
  template=prompt = f"""
  use the following CONTEXT to answer the QUESTION at the end.
  If you don't know the answer, just say that "I don't know", don't try to make up an answer.
  Also remember dont use any external knowledge and only refer to this CONTEXT to answer question.
  Consider this CONTEXT as the ultimate truth.

  CONTEXT: {CONTEXT}
  QUESTION: {query}

  """
  prompt = PromptTemplate(
      input_variables=["query","CONTEXT"],
      template=template
  )

  chain = LLMChain(llm=llm, prompt=prompt)
  response = chain.invoke({"query": query, "CONTEXT":CONTEXT})
  return response['text']

In [50]:
query = "Who created Batman comic"
CONTEXT = get_context(preprocessed_docs,query)
answer = retrieve(CONTEXT,query)
print(answer)

Batman was created by the artist Bob Kane and writer Bill Finger.


In [51]:
query = "Who was batmans girlfriend"
CONTEXT = get_context(preprocessed_docs,query)
answer = retrieve(CONTEXT,query)
print(answer)

The context does not provide information about Batman's girlfriend. Batman's romantic relationships are not mentioned or discussed in the given text.


In [52]:
query = "Any idea on Batman's love life"
CONTEXT = get_context(preprocessed_docs,query)
answer = retrieve(CONTEXT,query)
print(answer)

Based on the provided context, Batman's love life is quite complex and spans many decades. His first love interest was Julie Madison, and over the years, he has had many significant relationships. Vicki Vale, a journalist, is another important figure in his love life. However, her attempts to uncover Batman's true identity led to a complicated romantic involvement that waxed and waned over the years.

Selina Kyle, also known as Catwoman, is perhaps the most notable figure in Batman's romantic history. Their relationship is characterized by a blend of romance and rivalry. Despite the challenges, their love story resulted in the birth of Damian Wayne, who would grow to become the latest Robin and add a new layer of complexity to Batman's character.

Batman was also engaged to Talia al Ghul, but he had to break off the engagement due to her father's criminal ambitions and the conflict it caused in their relationship. His relationship with Wonder Woman has also been explored in various sto

In [54]:
query = "Who was Batman's most ferocious enemy?"
CONTEXT = get_context(preprocessed_docs,query)
answer = retrieve(CONTEXT,query)
print(answer)

Based on the provided context, it is not explicitly stated who Batman's most ferocious enemy is. However, Ghost-Maker, Scarecrow, Bane, and the Magistrate led by Simon Saint are all mentioned as enemies of Batman. Between these, Bane is depicted as a particularly formidable and physically powerful foe. In the provided context, it is also mentioned that Bane travels to Gotham after healing from his wounds to fight Batman, indicating a persistent and tenacious enmity. Therefore, based on the given information, it could be argued that Bane is one of Batman's more ferocious enemies. However, it is important to note that the context does not provide enough information to definitively answer this question.
